# Make an app notebook tutorial/pipeline

In [0]:
%pip install -U flask
dbutils.library.restartPython()

In [0]:
import os 
import base64
import openai
from openai import AzureOpenAI  
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [0]:
print(openai.__version__)

In [0]:
# #: store your AOAI API KEY in databricks secret vault
# databricks secrets put-secret yyang_secret_scope AZURE_OPENAI_API_KEY
# then use your editor to input your key string value and save.

In [0]:
with open('index.html', 'w') as file:
    file.write('''<!doctype html>
<head>
    <title>Hello Azure - Python Quickstart</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='bootstrap/css/bootstrap.min.css') }}">
    <link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}">
</head>
<html>
   <body>
     <main>
        <div class="px-4 py-3 my-2 text-center">
            <img class="d-block mx-auto mb-4" src="{{ url_for('static', filename='images/azure-icon.svg') }}" alt="Azure Logo" width="192" height="192"/>
            <!-- <img  src="/docs/5.1/assets/brand/bootstrap-logo.svg" alt="" width="72" height="57"> -->
            <h1 class="display-6 fw-bold text-primary">Welcome to Azure</h1>            
          </div>
        <form method="post" action="{{url_for('hello')}}">
            <div class="col-md-6 mx-auto text-center">
                <label for="req" class="form-label fw-bold fs-5">Input query below:</label>

                <!-- <p class="lead mb-2">Could you please tell me your name?</p> -->
                <div class="d-grid gap-2 d-sm-flex justify-content-sm-center align-items-center my-1">
                    <input type="text" class="form-control" id="req" name="req" style="max-width: 456px;">
                  </div>            
                <div class="d-grid gap-2 d-sm-flex justify-content-sm-center my-2">
                  <button type="submit" class="btn btn-primary btn-lg px-4 gap-3">Submit Request</button>
                </div>            
            </div>
        </form>
     </main>      
   </body>
</html>''')

In [0]:
with open('hello.html', 'w') as file:
    file.write('''<!doctype html>
<head>
    <title>Hello Azure - Python Quickstart</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='bootstrap/css/bootstrap.min.css') }}">
    <link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}">
</head>
<html>
   <body>
     <main>
        <div class="px-4 py-3 my-2 text-center">
            <img class="d-block mx-auto mb-4" src="{{ url_for('static', filename='images/azure-icon.svg') }}" alt="Azure Logo" width="192" height="192"/>
            <!-- <img  src="/docs/5.1/assets/brand/bootstrap-logo.svg" alt="" width="72" height="57"> -->
            <h1 class="display-6 fw-bold">OpenAI response:</h1>
            <p class="fs-5">
                {{req}}
            </p>
            <a href="{{ url_for('index') }}" class="btn btn-primary btn-lg px-4 gap-3">Back home</a>
          </div>
     </main>      
   </body>
</html>''')

In [0]:
# TODO: Replace the following with your own values and use Databricks secret vault instead of plain text.
# # Azure OpenAI
# api_key = os.environ['API_KEY']
# api_version = os.environ['API_VERSION']
# azure_deployment = os.environ['AZURE_DEPLOYMENT']
# model_name = os.environ['MODEL_NAME']

# https://portal.azure.com/#@DataBricksInc.onmicrosoft.com/resource/subscriptions/3f2e4d32-8e8d-46d6-82bc-5bb8d962328b/resourceGroups/fe-shared-amer-001/providers/Microsoft.CognitiveServices/accounts/DFCI-AOAI-Test/overview
# Azure OpenAI
api_key = os.environ['AZURE_OPENAI_API_KEY']
# api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-10-01" # from Resource JSON in the portal of the specific Azure OpenAI page.
api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-05-01-preview" # from Chat playground Sample Code (key authentication)
azure_deployment = os.environ['AZURE_OPENAI_ENDPOINT'] = "https://dfci-aoai-test.openai.azure.com/"
model_name = os.environ['AZURE_OPENAI_MODEL_NAME'] = "gpt-4o"

In [0]:
# TODO: Replace UI create way with mlflow api way

# from mlflow.deployments import get_deploy_client

# client = get_deploy_client("databricks")
# endpoint = client.create_endpoint(
#     name="chat",
#     config={
#         "served_entities": [
#             {
#                 "name": "chat",
#                 "external_model": {
#                     "name": "gpt-4o",
#                     "provider": "Azure openai",
#                     "task": "llm/v1/chat",
#                     "openai_config": {
#                         "openai_api_key": "{{secrets/scope/key}}",
#                     },
#                 },
#             }
#         ],
#     },
# )
# assert endpoint == {
#     "name": "chat",
#     "creator": "alice@company.com",
#     "creation_timestamp": 0,
#     "last_updated_timestamp": 0,
#     "state": {...},
#     "config": {...},
#     "tags": [...],
#     "id": "88fd3f75a0d24b0380ddc40484d7a31b",
# }


In [0]:
#: skip this, not needed
# %python
# import os

# os.environ["AZURE_CLIENT_ID"] = "your-client-id"
# os.environ["AZURE_TENANT_ID"] = "your-tenant-id"
# os.environ["AZURE_CLIENT_SECRET"] = "your-client-secret"

In [0]:
# endpoint = os.getenv("ENDPOINT_URL", "https://dfci-aoai-test.openai.azure.com/")  
# deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")  
      
# # Initialize Azure OpenAI Service client with Entra ID authentication
# token_provider = get_bearer_token_provider(  
#     DefaultAzureCredential(),  
#     "https://cognitiveservices.azure.com/.default"  
# )  
  
# client = AzureOpenAI(  
#     azure_endpoint=endpoint,  
#     azure_ad_token_provider=token_provider,  
#     api_version="2024-10-01",  
# )

# Alternatively, we don't use managed Azure identity, and use API key authentication.
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [0]:
# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

user_input = "Could you recommend 4 different stocks across 4 different sectors to buy? Please include tech, healthcare, finance, and energy."

chat_prompt = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "Scan the stock market to recommend stocks with strong fundamentals and technical indicators that indicate a good buy price. Provide comprehensive analysis incorporating fundamental data, technical indicators, and market sentiment. Given the complexity of this task, prioritize delivering depth over the total number of stock analyses offered. Recommended output can reasonably focus on 2-3 stocks."
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"{user_input}"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": ""
            }
        ]
    }
] 


chat_prompt = chat_prompt

# Include speech result if speech is enabled  
messages = chat_prompt 

completion = client.chat.completions.create(  
    model=model_name,  
    messages=messages,
    max_tokens=4096,  
    temperature=0.7,  
    top_p=0.95,  
    frequency_penalty=0,  
    presence_penalty=0,
    stop=None,  
    stream=False  
)  
  
print(completion.to_json())

## Create an Azure App

In [0]:
# make sure we have flask >3.0. In runtime ML 15.4, it is old version of flask, so will fail.
from flask import Flask, request

app = Flask(__name__)

@app.route('/hello', methods=['POST'])
def hello():
   req = request.form.get('req')

   llm = AzureOpenAI(
       api_key=api_key,
       api_version=api_version,
       azure_deployment=azure_deployment,
       model_name=model_name,
   )
   text = llm.invoke(req)
   return text

In [0]:
# import os

# # OpenAI
# from langchain_openai import ChatOpenAI

# # Azure OpenAI
# from langchain_openai import AzureOpenAI

In [0]:
# based on above cell, write an web app deployable to Azure App Service via Git repo.
import os
import base64
from flask import Flask, request, jsonify
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

app = Flask(__name__)

endpoint = os.getenv("ENDPOINT_URL", "https://dfci-aoai-test.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")

@app.route('/analyze_stocks', methods=['POST'])
def analyze_stocks():
    data = request.json
    chat_prompt = [
        {
            "role": "system",
            "content": data.get("system_content", "")
        },
        {
            "role": "user",
            "content": data.get("user_content", "")
        },
        {
            "role": "assistant",
            "content": data.get("assistant_content", "")
        }
    ]

    messages = chat_prompt

    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=4096,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )

    return jsonify(completion.to_json())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [0]:
%pip install -U flask

In [0]:
%restart_python

In [0]:
from flask import Flask, render_template_string

app = Flask(__name__)

html_code = """
<!doctype html>
<head>
    <title>Hello Azure - Python Quickstart</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='bootstrap/css/bootstrap.min.css') }}">
    <link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}">
</head>
<html>
   <body>
     <main>
        <div class="px-4 py-3 my-2 text-center">
            <img class="d-block mx-auto mb-4" src="{{ url_for('static', filename='images/azure-icon.svg') }}" alt="Azure Logo" width="192" height="192"/>
            <h1 class="display-6 fw-bold text-primary">Welcome to Azure</h1>            
          </div>
        <form method="post" action="{{url_for('hello')}}">
            <div class="col-md-6 mx-auto text-center">
                <label for="name" class="form-label fw-bold fs-5">Could you please tell me your name?</label>
                <div class="d-grid gap-2 d-sm-flex justify-content-sm-center align-items-center my-1">
                    <input type="text" class="form-control" id="name" name="name" style="max-width: 256px;">
                  </div>            
                <div class="d-grid gap-2 d-sm-flex justify-content-sm-center my-2">
                  <button type="submit" class="btn btn-primary btn-lg px-4 gap-3">Say Hello</button>
                </div>            
            </div>
        </form>
     </main>      
   </body>
</html>
"""

@app.route('/')
def preview():
    return render_template_string(html_code)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [0]:
import os
from flask import Flask, request, jsonify
from openai import AzureOpenAI

app = Flask(__name__)

api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://your-endpoint.openai.azure.com/")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME", "gpt-4o")

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

@app.route('/analyze_stocks', methods=['POST'])
def analyze_stocks():
    data = request.json
    chat_prompt = [
        {
            "role": "system",
            "content": data.get("system_content", "")
        },
        {
            "role": "user",
            "content": data.get("user_content", "")
        },
        {
            "role": "assistant",
            "content": data.get("assistant_content", "")
        }
    ]

    messages = chat_prompt

    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=4096,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )

    return jsonify(completion.to_json())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

In [0]:
import os 
import base64
import openai
from openai import AzureOpenAI  
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
print(openai.__version__)



# Azure OpenAI
api_key = os.environ['AZURE_OPENAI_API_KEY']
# api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-10-01" # from Resource JSON in the portal of the specific Azure OpenAI page.
api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-05-01-preview" # from Chat playground Sample Code (key authentication)
azure_deployment = os.environ['AZURE_OPENAI_ENDPOINT'] = "https://dfci-aoai-test.openai.azure.com/"
model_name = os.environ['AZURE_OPENAI_MODEL_NAME'] = "gpt-4o"


# Alternatively, we don't use managed Azure identity, and use API key authentication.
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )


# make sure we have flask >3.0. In runtime ML 15.4, it is old version of flask, so will fail.
from flask import (Flask, redirect, render_template, request, send_from_directory, url_for)

app = Flask(__name__)


@app.route('/')
def index():
   print('Request for index page received')
   return render_template('index.html')

@app.route('/favicon.ico')
def favicon():
    return send_from_directory(os.path.join(app.root_path, 'static'),
                               'favicon.ico', mimetype='image/vnd.microsoft.icon')

@app.route('/hello', methods=['POST'])
def hello():
   req = request.form.get('req')

   # Azure OpenAI
   llm = AzureOpenAI(
       api_key=api_key,
       api_version=api_version,
       azure_deployment=azure_deployment,
       model_name=model_name,
   )
   text = llm.invoke(req)

   # OpenAI
	 # llm = ChatOpenAI(openai_api_key=openai_api_key)
	 # text = llm.invoke(req)

   if req:
       print('Request for hello page received with req=%s' % req)
       return render_template('hello.html', req = text)
   else:
       print('Request for hello page received with no name or blank name -- redirecting')
       return redirect(url_for('index'))
   
if __name__ == '__main__':
   app.run(host='0.0.0.0', port=5000)



## Test app locally

In [0]:
%pip install -U openai azure-identity

In [0]:
%pip install -U flask

In [0]:
dbutils.library.restartPython()

In [0]:
%python
%run -i app.py

In [0]:
import os
import base64
from flask import Flask, request, jsonify
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

app = Flask(__name__)

endpoint = os.getenv("ENDPOINT_URL", "https://dfci-aoai-test.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")

@app.route('/analyze_stocks', methods=['POST'])
def analyze_stocks():
    data = request.json
    chat_prompt = [
        {
            "role": "system",
            "content": data.get("system_content", "")
        },
        {
            "role": "user",
            "content": data.get("user_content", "")
        },
        {
            "role": "assistant",
            "content": data.get("assistant_content", "")
        }
    ]

    messages = chat_prompt

    completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        max_tokens=4096,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )

    return jsonify(completion.to_json())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

In [0]:
!flask run --host=0.0.0.0 --port=5000

In [0]:
%sh
cd msdocs-python-flask-webapp-quickstart
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
flask run


# New Tests

In [0]:
%pip install -U flask

In [0]:
%pip install --upgrade langchain_openai
%pip install --upgrade langchain langchain-community langchain-core



In [0]:
%restart_python

In [0]:
import os
from flask import Flask, request, jsonify, render_template
from openai import AzureOpenAI

app = Flask(__name__)

api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://dfci-aoai-test.openai.azure.com/")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME", "gpt-4o")

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=azure_endpoint
)

@app.route('/')
def index():
    print('Request for index page received')
    return render_template('index.html')
  
@app.route('/aoai_query')
def aoai_query():
    print('Request for aoai_query page received')
    return render_template('aoai_query.html')
  

@app.route('/analyze_stocks', methods=['POST'])
def analyze_stocks():
    data = request.form
    chat_prompt = [
        {
            "role": "system",
            "content": data.get("system_content", "")
        },
        {
            "role": "user",
            "content": data.get("user_content", "")
        },
        {
            "role": "assistant",
            "content": data.get("assistant_content", "")
        }
    ]

    messages = chat_prompt

    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=4096,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False
    )

    return jsonify(completion.to_json())

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False)

In [0]:
import os
# Azure OpenAI
from langchain_openai import AzureOpenAI


from flask import (Flask, redirect, render_template, request,
                   send_from_directory, url_for)

app = Flask(__name__)

@app.route('/')
def index():
   print('Request for index page received')
   return render_template('index.html')

@app.route('/favicon.ico')
def favicon():
    return send_from_directory(os.path.join(app.root_path, 'static'),
                               'favicon.ico', mimetype='image/vnd.microsoft.icon')

# Azure OpenAI
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-05-01-preview")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://dfci-aoai-test.openai.azure.com/")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME", "gpt-4o")

# OpenAI
# openai_api_key = os.environ['OPENAI_API_KEY']

@app.route('/hello', methods=['POST'])
def hello():
   req = request.form.get('req')

   # Azure OpenAI
   llm = AzureOpenAI(
       api_key=api_key,
       api_version=api_version,
       azure_deployment=azure_deployment,
       model_name=model_name,
   )
   text = llm.invoke2(req)

   # OpenAI
	 # llm = ChatOpenAI(openai_api_key=openai_api_key)
	 # text = llm.invoke(req)

   if req:
       print('Request for hello page received with req=%s' % req)
       return render_template('hello.html', req = text)
   else:
       print('Request for hello page received with no name or blank name -- redirecting')
       return redirect(url_for('index'))

if __name__ == '__main__':
   app.run()

In [0]:
app = Flask(__name__)
if __name__ == '__main__':
   app.run()

In [0]:
%sh flask run --host=0.0.0.0 --port=5000

In [0]:
# import os 
# import base64
# import openai
# from openai import AzureOpenAI  
# from azure.identity import DefaultAzureCredential, get_bearer_token_provider
# print(openai.__version__)

# # Azure OpenAI
# api_key = os.environ['AZURE_OPENAI_API_KEY'] = dbutils.secrets.get(scope="yyang_secret_scope", key="AZURE_OPENAI_API_KEY")
# # api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-10-01" # from Resource JSON in the portal of the specific Azure OpenAI page.
# api_version = os.environ['AZURE_OPENAI_API_VERSION'] = "2024-05-01-preview" # from Chat playground Sample Code (key authentication)
# azure_deployment = os.environ['AZURE_OPENAI_ENDPOINT'] = "https://dfci-aoai-test.openai.azure.com/"
# model_name = os.environ['AZURE_OPENAI_MODEL_NAME'] = "gpt-4o"


# # Alternatively, we don't use managed Azure identity, and use API key authentication.
# client = AzureOpenAI(
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
#     api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
#     azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
#     )


# # make sure we have flask >3.0. In runtime ML 15.4, it is old version of flask, so will fail.
# from flask import (Flask, redirect, render_template, request, send_from_directory, url_for)

# app = Flask(__name__)


# @app.route('/')
# def index():
#    print('Request for index page received')
#    return render_template('index.html')

# @app.route('/favicon.ico')
# def favicon():
#     return send_from_directory(os.path.join(app.root_path, 'static'),
#                                'favicon.ico', mimetype='image/vnd.microsoft.icon')

# @app.route('/hello', methods=['POST'])
# def hello():
#    req = request.form.get('req')

#    # Azure OpenAI
#    llm = AzureOpenAI(
#        api_key=api_key,
#        api_version=api_version,
#        azure_deployment=azure_deployment,
#        model_name=model_name,
#    )
#    text = llm.invoke(req)

#    # OpenAI
# 	 # llm = ChatOpenAI(openai_api_key=openai_api_key)
# 	 # text = llm.invoke(req)

#    if req:
#        print('Request for hello page received with req=%s' % req)
#        return render_template('hello.html', req = text)
#    else:
#        print('Request for hello page received with no name or blank name -- redirecting')
#        return redirect(url_for('index'))
   
# if __name__ == '__main__':
#    app.run()

